In [ ]:
#outline of project
#grab stock info from api

#create website that shares each stock, dropdown menu to click a specific stock and graphs and charts with that specific one

In [16]:
from apikey import API
import pandas as pd
import numpy as np
import matplotlib_inline
import requests, time, re, os
import pickle as pkl

In [3]:
#address to the api
url= "https://api.tdameritrade.com/v1/instruments"

In [7]:
#load df that has all the symbol names of stocks
df=pd.read_csv("stocks-list.csv")
df.head()

,Symbol,Company Name,Industry,Market Cap
0,A,Agilent Technologies,Life Sciences Tools & Services,4.571910e+10
1,AA,Alcoa,Metals & Mining,8.869851e+09
2,AAC,Ares Acquisition,Blank Check / SPAC,1.252500e+09
3,AACG,ATA Creativity Global,Diversified Consumer Services,5.490958e+07
4,AACI,Armada Acquisition I,Blank Check / SPAC,2.068879e+08


In [9]:
#get symbol column only
symbols=df['Symbol'].values.tolist()

In [19]:
#loop to iterate through all the symbols and info
start=0
end=500
files=[]

while start < len(symbols):
    tickers=symbols[start:end]
    
    payload={'apikey': API,
        'symbol': tickers,
        'projection':'fundamental'}

    results=requests.get(url, params=payload)

    data=results.json()
    file_name='stocks_'+ time.asctime()+ '.pkl'
    file_name=re.sub('[ :]', '_', file_name)
    files.append(file_name)
    with open(file_name, 'wb') as file:
        pkl.dump(data,file)
    start=end
    end+=500
    time.sleep(1)


In [20]:
#grab only the specific key value pairs for each symbol
data=[]

for file in files:
    with open(file,'rb') as f:
        info=pkl.load(f)
    
    tickers=list(info)
    points=['symbol','netProfitMarginMRQ','peRatio','pegRatio','high52']
    for ticker in tickers:
        tick=[]
        for point in points:
            tick.append(info[ticker]['fundamental'][point])
        data.append(tick)
    
    os.remove(file)

In [21]:
#column names
columns=['symbol','Margin','PE','PEG','high52']

#add to df
df_results=pd.DataFrame(data,columns=columns)


In [23]:
df_results

,symbol,Margin,PE,PEG,high52
0,AGLE,0.00000,0.00000,0.000000,6.37
1,AEHA,0.00000,0.00000,0.000000,10.60
2,AJRD,2.49182,45.63846,0.000000,56.00
3,ADEX,0.00000,133.85830,0.000000,10.03
4,ADER,0.00000,39.34478,0.000000,10.06
...,...,...,...,...,...
6167,XAIR,0.00000,0.00000,0.000000,14.75
6168,XEL,15.89907,22.92973,4.731145,77.66
6169,XPON,0.00000,0.00000,0.000000,11.29
6170,YOU,0.00000,0.00000,0.000000,35.08


In [24]:
#filter for the best stocks rn
best_stocks_df=df_results[(df_results['PEG'] <1) & (df_results['PEG'] >0) & (df_results['Margin'] >20) & (df_results['PE'] >10)]
best_stocks_df

#sort this list by best WHAT to get the best stocks at the top

,symbol,Margin,PE,PEG,high52
21,ABBV,26.68107,21.47180,0.272895,175.910
93,ADI,28.82721,32.61579,0.758331,188.195
157,AHH,34.51172,15.03415,0.046241,15.800
174,AAIC,21.81737,46.36974,0.321827,3.740
192,AMAT,23.57386,14.73606,0.922576,167.060
...,...,...,...,...,...
6053,YNDX,34.19944,14.35695,0.037012,73.580
6061,ZTO,21.33960,22.53026,0.431540,32.670
6102,WPM,89.77499,22.18811,0.795293,51.905
6140,WT,112.17310,11.10380,0.032227,6.600


In [25]:
#def view as listing the top amount of stocks you want 
def view(size):
    start=0
    end=size
    while end < len(best_stocks_df):
        print(best_stocks_df[start:end])
        start=end
        end += size
    print(best_stocks_df[start:end])

In [26]:
#check code
view(10)

    symbol    Margin        PE       PEG    high52
21    ABBV  26.68107  21.47180  0.272895  175.9100
93     ADI  28.82721  32.61579  0.758331  188.1950
157    AHH  34.51172  15.03415  0.046241   15.8000
174   AAIC  21.81737  46.36974  0.321827    3.7400
192   AMAT  23.57386  14.73606  0.922576  167.0600
198   AMAL  31.58982  11.45943  0.260611   27.3300
205   ACMR  20.24995  18.58805  0.589605   30.5300
208   ABMD  39.91148  64.93067  0.822962  381.9900
221    ALB  32.14076  21.07861  0.035269  334.5499
229   ANET  30.08147  37.95423  0.773727  148.5700
    symbol     Margin        PE       PEG   high52
235   ABNB   42.09831  42.61551  0.320050  191.730
239    ALX   28.12075  10.44354  0.166387  274.460
323   AMGN   32.21587  22.94583  0.803623  296.670
352   ALGM   21.31058  50.16512  0.593709   38.281
579   BIPC   72.90749  20.01084  0.119865   53.640
594   BFIN   21.17825  13.76115  0.360142   11.270
624    BHB   27.97669  10.96845  0.257579   31.950
674   CAAP   21.52494  17.91836

In [30]:
#add industry name to the stocks
df_symbols= best_stocks_df['symbol'].tolist()
bools=df['Symbol'].isin(df_symbols)
industry=df[bools]
industry

,Symbol,Company Name,Industry,Market Cap
6,AAIC,Arlington Asset Investment,Mortgage Real Estate Investment Trust...,8.623477e+07
22,ABBV,AbbVie,Biotechnology,2.850440e+11
33,ABMD,Abiomed,Health Care Equipment & Supplies,1.700975e+10
34,ABNB,Airbnb,"Hotels, Restaurants & Leisure",6.467270e+10
72,ACMR,ACM Research,Semiconductors & Semiconductor Equipment,5.387103e+08
...,...,...,...,...
6024,WPM,Wheaton Precious Metals,Metals & Mining,1.704195e+10
6043,WT,WisdomTree,Asset Management,8.161109e+08
6091,XP,XP Inc.,Capital Markets,9.810748e+09
6123,YNDX,Yandex,Interactive Media & Services,6.797282e+09
